<a href="https://colab.research.google.com/github/john-caballero/Data-Discovery/blob/main/modulo3_manipulacion_datos_faltantes/Handling_Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://posgrado.utec.edu.pe/sites/default/files/2023-08/Testimonial-home-2.jpg" alt="HTML5 Icon" width="900" height="250" >


# **Handling Missing Data**

**Objetivos**

- Aplicar distintas técnicas de imputación:

  - Medidas de tendencia central (media, mediana, moda)

  - Imputación por regresión

  - KNN imputation

  - XGBoost imputador

  - Autoencoders

  - MICE (Multiple Imputation by Chained Equations)

- Entrenar un modelo base (RandomForest o XGBoost) para comparar el impacto de cada estrategia de imputación en términos de performance (accuracy, AUC, etc.)

- Reflexionar sobre las implicancias de cada método para modelos analíticos.

**Dataset**

Loan Prediction - train.csv

Puedes obtenerlo desde:
 https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii

Librerías necesarias

In [ ]:
!pip install pandas numpy seaborn matplotlib scikit-learn xgboost fancyimpute tensorflow missingno


### 1. Carga y diagnóstico inicial

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")
df.drop("Loan_ID", axis=1, inplace=True)


### 2. Medidas de tendencia central (media, mediana, moda)

In [ ]:
df_mean = df.copy()

df_mean["LoanAmount"].fillna(df_mean["LoanAmount"].mean(), inplace=True)
df_mean["Gender"].fillna(df_mean["Gender"].mode()[0], inplace=True)
df_mean["Self_Employed"].fillna(df_mean["Self_Employed"].mode()[0], inplace=True)
df_mean["Dependents"].fillna(df_mean["Dependents"].mode()[0], inplace=True)
df_mean["Credit_History"].fillna(df_mean["Credit_History"].mode()[0], inplace=True)


### 3. Imputación por regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

df_reg = df.copy()
train = df_reg[df_reg["LoanAmount"].notnull()]
test = df_reg[df_reg["LoanAmount"].isnull()]

X_train = train[["ApplicantIncome", "CoapplicantIncome"]]
y_train = train["LoanAmount"]
X_test = test[["ApplicantIncome", "CoapplicantIncome"]]

model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

df_reg.loc[df_reg["LoanAmount"].isnull(), "LoanAmount"] = preds


### 4. 🔎 Imputación KNN

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

df_knn = df.copy()
df_knn = df_knn.apply(LabelEncoder().fit_transform)

knn_imputer = KNNImputer(n_neighbors=5)
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)


### 5. Imputación con XGBoost (modelo para imputar)

In [ ]:
import xgboost as xgb

df_xgb = df.copy()

def xgb_impute(df, target_col):
    train = df[df[target_col].notnull()]
    test = df[df[target_col].isnull()]
    features = [col for col in df.columns if col != target_col and df[col].notnull().sum() > 0]

    train = train.dropna(subset=features)
    X_train = pd.get_dummies(train[features])
    y_train = train[target_col]
    X_test = pd.get_dummies(test[features])
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    model = xgb.XGBRegressor(n_estimators=100)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    df.loc[df[target_col].isnull(), target_col] = preds
    return df

df_xgb = xgb_impute(df_xgb, "LoanAmount")


### 6. Imputación MICE (Multivariate Imputation)

In [ ]:
from fancyimpute import IterativeImputer

df_mice = df.copy()
df_mice = df_mice.apply(LabelEncoder().fit_transform)

mice = IterativeImputer()
df_mice_imputed = pd.DataFrame(mice.fit_transform(df_mice), columns=df_mice.columns)


### 7. Autoencoders para imputación

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df_auto = df.copy()
df_auto = df_auto.apply(LabelEncoder().fit_transform)
df_auto = df_auto.fillna(df_auto.mean())

X = df_auto.values

model = Sequential([
    Dense(32, activation='relu', input_shape=(X.shape[1],)),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(X.shape[1])
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, X, epochs=100, batch_size=16, verbose=0)

X_pred = model.predict(X)
df_auto_imputed = pd.DataFrame(X_pred, columns=df_auto.columns)


### 8. Evaluación del impacto predictivo

Utilizaremos un modelo de clasificación (XGBClassifier) para predecir Loan_Status con los diferentes datasets imputados.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier

def evaluate_model(df, name=""):
    df = df.copy()
    df = df.dropna()
    X = df.drop("Loan_Status", axis=1)
    y = df["Loan_Status"]

    if y.dtype == 'O':
        y = LabelEncoder().fit_transform(y)

    X = pd.get_dummies(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    probas = model.predict_proba(X_test)[:,1]

    print(f"🔍 Evaluación con {name}")
    print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
    print(f"AUC: {roc_auc_score(y_test, probas):.4f}\n")

# Evaluar
evaluate_model(df_mean, "Media/Moda")
evaluate_model(df_reg, "Regresión")
evaluate_model(df_knn_imputed, "KNN")
evaluate_model(df_xgb, "XGBoost")
evaluate_model(df_mice_imputed, "MICE")
evaluate_model(df_auto_imputed, "Autoencoder")


### Preguntas para reflexión

- ¿Qué técnica ofrece mejor balance entre simplicidad y precisión?

- ¿Qué riesgos podría implicar usar una técnica muy compleja como Autoencoders o XGBoost para imputar?

- ¿Por qué KNN puede ser sensible a la escala de los datos o a outliers?

- ¿Cómo se podría incorporar la incertidumbre de la imputación en el modelo final?

- ¿Qué tipo de datos (categóricos, numéricos, multivariados) favorecen el uso de MICE sobre otros métodos?

### Conclusión

La imputación no es solo un paso técnico: es una decisión analítica que puede alterar el resultado del modelo. Evaluar diferentes métodos no solo mejora el desempeño, sino también la confianza en los modelos desarrollados. Entender cuándo usar cada técnica y su impacto es clave para un análisis responsable.

---

# Gracias por completar este laboratorio!

---
